# A Critique of the Bounded Fuzzy Possibilistic Method 
## Iris Data Set

The *Bounded Fuzzy Possibilistic Method* (BFPM) was recently introduced in [doi:10.1016/j.fss.2019.07.011](https://dx.doi.org/10.1016/j.fss.2019.07.011). I demonstrate that there are some critical flaws in the proposed algorithm, which makes the results presented therein highly questionable. In particular, the method may generate meaningless cluster membership degrees or even fail to converge when run on some classical benchmark data sets.

In [1]:
import numpy as np
import pandas as pd
from bfpm import partition  # see bfpm.py in this repository — an implementation of the BFPM and Fuzzy c-means
np.random.seed(123)  # reproducibility matters

Let's consider the famous *Iris* data set that can be downloaded from the UCI Machine Learning repository (https://archive.ics.uci.edu/ml/).

In [2]:
# https://archive.ics.uci.edu/ml/machine-learning-databases/iris/
iris = pd.read_csv("iris.data", names=["x1", "x2", "x3", "x4", "y"])
iris_X = np.array(iris.iloc[:,:4])
iris_Y = np.array(iris.y.astype("category").cat.codes)
iris_c = len(np.unique(iris_Y))  # number of reference clusters

The data set `iris_X` consists of $n=150$ points in $\mathbb{R}^{4}$ and its reference partition `iris_Y` (given by the 5th column of `iris`) defines $c=3$ clusters.

 The 3 reference cluster centres are as follows:

In [3]:
iris_V_true = np.array(iris.groupby("y").mean())
print(iris_V_true)

[[5.006 3.418 1.464 0.244]
 [5.936 2.77  4.26  1.326]
 [6.588 2.974 5.552 2.026]]


Running the FCM with fuzzifier $m=2$ and random initial $V$ yields:

In [4]:
iris_V_FCM = partition(iris_X, iris_c, V_init=None, m=2.0, use_FCM=True)
iris_V_FCM

array([[6.7750278 , 3.05240438, 5.64680082, 2.05356469],
       [5.88912443, 2.76120781, 4.36414496, 1.39738975],
       [5.00356114, 3.40303928, 1.48499544, 0.25153832]])

The identified cluster centres are quite close to the reference ones (note that clusters are defined up to a permutation of labels, i.e., up to a permutation of the rows in the above matrix).

Let's now run the BFPM with $m=2$, with the initial $U$ being randomly generated.

In [5]:
iris_V_BFPM = partition(iris_X, iris_c,
    V_init=None, m=2.0, use_FCM=False, verbose=True)

--- cycle in V updates!
--- V in iteration 41:
[[5.1    3.4    1.5    0.2   ]
 [5.1    3.4    1.5    0.2   ]
 [6.2694 2.8896 4.9681 1.754 ]]
--- V in iteration 42:
[[6.2593 2.8974 4.929  1.7348]
 [6.2593 2.8974 4.929  1.7348]
 [5.1    3.4    1.5    0.2   ]]
--- V in iteration 43 (current):
[[5.1    3.4    1.5    0.2   ]
 [5.1    3.4    1.5    0.2   ]
 [6.2694 2.8896 4.9682 1.754 ]]


Unfortunately, the algorithm does not converge at all. If we hadn't added an extra stopping criterion, the BFPM would have run indefinitely. It turns out that the algorithm bounces between two stationary points, both representing the same (up to numeric error) two, not three, clusters. Note that two of the generated cluster centres coincide with each other.

The same happens when we initialise the BFPM with the true cluster centres:

In [6]:
iris_V_BFPM = partition(iris_X, iris_c,
    V_init=iris_V_true, m=2.0, use_FCM=False, verbose=True)

--- cycle in V updates!
--- V in iteration 29:
[[6.2696 2.8896 4.9683 1.7541]
 [5.1    3.4    1.5    0.2   ]
 [5.1    3.4    1.5    0.2   ]]
--- V in iteration 30:
[[5.1    3.4    1.5    0.2   ]
 [6.2594 2.8974 4.9291 1.7348]
 [6.2594 2.8974 4.9291 1.7348]]
--- V in iteration 31 (current):
[[6.2695 2.8896 4.9683 1.7541]
 [5.1    3.4    1.5    0.2   ]
 [5.1    3.4    1.5    0.2   ]]
